In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# Install required libs

### please update Albumentations to version>=0.3.0 for `Lambda` transform support
!pip install -U git+https://github.com/albu/albumentations --no-cache-dir

!pip uninstall -y opencv-python
!pip install opencv-python

In [3]:
# !git clone https://github.com/qubvel/segmentation_models.pytorch

In [ ]:
%cd ./gdrive/MyDrive/Colab Notebooks/Solar Panels

!pip install -r requirements.txt
!ls

# Loading dataset

In [5]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

import torch

import segmentation_models_pytorch as smp

from pathlib import Path

from torch.utils.data import DataLoader

print(smp.__version__)
# os.environ['CUDA_VISIBLE_DEVICES'] = '0'

0.2.1


# Dataloader and utility functions 

In [6]:
# from src.models.segmentation.datasets import *
from src.dataloader import *

In [7]:
from src.transformers import *

# Segmentation model training

In [8]:
import pytorch_lightning as pl
from src.model import SolarPanelsModel

In [9]:
def get_model_info(model_name):
    info, ext = model_name.split('.')
    arch, *enc, epochs = info.split('_')

    enc = '_'.join(enc[:-1])
    raw_name = arch + '_' + enc
    return raw_name, enc, int(epochs)

def model_exists(model_name):
    parent = Path(model_name).parent
    name, _, _ = get_model_info(model_name)
    for model in os.listdir(parent):
        if model.startswith(name):
            return os.path.join(parent, model)


In [10]:
def train(params, device, verbose=True):
    model = SolarPanelsModel(
        arch=params['architecture'],
        encoder=params['encoder'],
        in_channels=3,
        out_classes=len(params['classes']),
        model_params=params
    )
    sp_module = SolarPanelsDataModule(
        data_dir=params['data_dir'],
        classes=params['classes'],
        train_augmentation=params['train_augmentation'],
        valid_augmentation=params['valid_augmentation'],
        preprocessing=params['preprocessing'],
        batch_size=params['batch_size'],
        num_workers=params['num_workers'],
    )
    trainer = pl.Trainer(gpus=-1, max_epochs=params['epochs'])
    trainer.fit(model, datamodule=sp_module)

    valid_metrics = trainer.validate(model, datamodule=sp_module, verbose=False)

    return trainer, valid_metrics

In [11]:
def gen_params(arch, encoder, epochs, data_dir, results_dir):
    return {
        'data_dir': data_dir,
        'results_dir': results_dir,
        'model_name': f'{results_dir}/{arch.lower()}_{encoder}_e{epochs}.pth',

        'architecture': arch,
        'encoder': encoder,
        'classes': CLASSES,
        'lr': LR,
        'epochs': epochs,
        'batch_size': BATCH_SIZE,

        'train_augmentation': get_training_augmentation,
        'valid_augmentation': get_validation_augmentation,
        'preprocessing': get_preprocessing,
        'num_workers': 2,

        'loss': LOSS,
        'optimizer': torch.optim.Adam
    }

In [12]:
DEVICE = 'cuda'

CLASSES = ['solar_panel']
BATCH_SIZE = 16
LR = 0.0001
LOSS = smp.losses.DiceLoss(smp.losses.BINARY_MODE, from_logits=True)

DATA_DIR = 'data'
RESULTS_DIR = 'models_pytorch'

ARCHITECTURE = "UnetPlusPlus"
ENCODER = 'se_resnext101_32x4d'
EPOCHS = 5

In [13]:
model_params = gen_params(ARCHITECTURE, ENCODER, EPOCHS, DATA_DIR, RESULTS_DIR)

In [14]:
trainer, valid_metrics = train(model_params, device='cuda')

if not os.path.exists(model_params['model_name']):
    torch.save(trainer.model.state_dict(), model_params['model_name'])

Downloading: "http://data.lip6.fr/cadene/pretrainedmodels/se_resnext101_32x4d-3b2fe3d8.pth" to /root/.cache/torch/hub/checkpoints/se_resnext101_32x4d-3b2fe3d8.pth


  0%|          | 0.00/187M [00:00<?, ?B/s]

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


<function get_training_augmentation at 0x7f4551c37ef0>


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type         | Params
-----------------------------------------
0 | model   | UnetPlusPlus | 72.4 M
1 | loss_fn | DiceLoss     | 0     
-----------------------------------------
72.4 M    Trainable params
0         Non-trainable params
72.4 M    Total params
289.537   Total estimated model params size (MB)


...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


<function get_training_augmentation at 0x7f4551c37ef0>


Validating: 0it [00:00, ?it/s]

In [15]:
print(valid_metrics)

[{'valid_per_image_iou': 0.8144766688346863, 'valid_dataset_iou': 0.9060141444206238, 'valid_per_image_f1': 0.8639066815376282, 'valid_dataset_f1': 0.9506899118423462}]


In [ ]:
# from itertools import product

# epochs = 25
# architectures = [smp.Unet, smp.UnetPlusPlus, smp.MAnet, smp.Linknet, smp.FPN, smp.PSPNet, smp.PAN, smp.DeepLabV3, smp.DeepLabV3Plus]

# encoders = [
#     # 'resnet50',
#     # 'resnext50_32x4d',
#     # 'timm-resnest50d_4s2x40d',
#     # 'timm-res2next50', 'timm-regnetx_064', 'timm-gernet_m',
#     # 'se_resnext101_32x4d',
#     # 'densenet201',
#     # 'xception',
#     'efficientnet-b2',
#     # 'timm-efficientnet-b3',
#     'timm-mobilenetv3_large_100',
#     'vgg16_bn', 'vgg19_bn'
# ]
# count = 0
# for arch, encoder in product(architectures, encoders):
#     train_params = gen_params(arch, encoder, epochs)
#     print('ARCH:', train_params['architecture'].__name__)
#     print('ENCODER:', train_params['encoder'])

#     train(train_params, DEVICE, verbose=False)

#     best_model = torch.load(train_params['model_name'])
#     _, logs = test(best_model, train_params, DEVICE)

#     if logs.get('fscore') > 0.92:
#         break

#     print()

# Model Evaluation

In [14]:
def load_model(model_path, params):
    model = SolarPanelsModel(
        arch=params['architecture'],
        encoder=params['encoder'],
        in_channels=3,
        out_classes=len(params['classes']),
        model_params=params
    )
    model.load_state_dict(torch.load(model_path), strict=False)
    return model

In [15]:
def test(model_path, params):

    model = load_model(model_path, params)

    datamodule = SolarPanelsDataModule(
        data_dir=params['data_dir'],
        classes=params['classes'],
        train_augmentation=params['train_augmentation'],
        valid_augmentation=params['valid_augmentation'],
        preprocessing=params['preprocessing'],
        batch_size=params['batch_size'],
        num_workers=params['num_workers'],
    )
    trainer = pl.Trainer(gpus=-1)
    test_metrics = trainer.test(model, datamodule=datamodule, verbose=False)

    return trainer, test_metrics

In [22]:
_, metrics = test('models_pytorch/unetplusplus_se_resnext101_32x4d_e5.pth', model_params)
print(metrics)

Downloading: "http://data.lip6.fr/cadene/pretrainedmodels/se_resnext101_32x4d-3b2fe3d8.pth" to /root/.cache/torch/hub/checkpoints/se_resnext101_32x4d-3b2fe3d8.pth


  0%|          | 0.00/187M [00:00<?, ?B/s]

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


dm
hi
<function get_training_augmentation at 0x7fe494563ef0>


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

[{'test_per_image_iou': 0.8243892192840576, 'test_dataset_iou': 0.9065484404563904, 'test_per_image_f1': 0.8724713921546936, 'test_dataset_f1': 0.9509838819503784}]
